In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [4]:
trainFile=pd.read_csv("gender_age_train.csv")
testFile=pd.read_csv("gender_age_test.csv")

In [5]:
#print(trainFile.shape)#(74654,4)
trainFile[0:1]

,device_id,gender,age,group
0,-8076087639492063270,M,35,M32-38


In [6]:
#print(testFile.shape)#(112071,1)
testFile[0:1]

,device_id
0,1002079943728939269


In [7]:
#Understanding Target Value Characteristics(Regression w.r.t. Age, Classification w.r.t. Gender)
trainGrpBy=trainFile.groupby("group")
trainGroups=trainGrpBy.groups
trainGroupDict=[(key,len(trainGroups[key])) for key in trainGroups.keys()]
print(trainGroupDict)
#Female Groups are : F23-,F24-26,F27-28,F29-32,F33-42,F43+
#Male Groups are : M22-,M23-26,M27-28,M29-31,M32-38,M39+

[('M29-31', 7309), ('F43+', 4194), ('M27-28', 5445), ('M39+', 8581), ('F29-32', 4628), ('M32-38', 9476), ('F33-42', 5561), ('F24-26', 4190), ('F23-', 5050), ('M22-', 7488), ('F27-28', 3118), ('M23-26', 9605)]


In [8]:
len(trainFile.groupby('device_id').groups.keys())#Number of unique device_ids: 74645 i.e. All device_id are unique

74645

In [9]:
trainFile.describe()#Interesting we have a Kid of Age 1 and grandpa having age 96 using Mobile Device :-)
#But Most of the users are withing age group 25 to 36

,device_id,age
count,7.464500e+04,74645.000000
mean,-7.491354e+14,31.410342
std,5.327150e+18,9.868735
min,-9.223067e+18,1.000000
25%,-4.617367e+18,25.000000
50%,-1.841362e+16,29.000000
75%,4.636656e+18,36.000000
max,9.222849e+18,96.000000


In [10]:
eventsData=pd.read_csv("events_subset.csv")
print(eventsData[0:2])
#eventsData.describe()
#event_id is PK here;device_id may get duplicated
#longitude: -180 to 151 : Continuous
#latitude: -37 to 53.64 : Continuous

   event_id            device_id            timestamp  longitude  latitude
0         1    29182687948017175  2016-05-01 00:55:25     121.38     31.24
1         2 -6401643145415154744  2016-05-01 00:54:12     103.65     30.97


In [11]:
#As expected => Very High Correlation between longitude and latitude : 0.947287
eventsData.corr()

,event_id,device_id,longitude,latitude
event_id,1.000000,0.001050,-0.003879,-0.004633
device_id,0.001050,1.000000,-0.013765,-0.007659
longitude,-0.003879,-0.013765,1.000000,0.947287
latitude,-0.004633,-0.007659,0.947287,1.000000


In [12]:
app_eventsData=pd.read_csv("app_events_subset.csv")

In [13]:
app_eventsData[0:1]

,event_id,app_id,is_installed,is_active
0,2,5927333115845830913,1,1


In [14]:
appDataGrpByObj=app_eventsData.groupby('app_id')
eventsDataGrpByObj=app_eventsData.groupby('event_id')
isActiveGrpByObj=app_eventsData.groupby('is_active')
isInstalledGrpByObj=app_eventsData.groupby('is_installed')
app_event_GrpByObj=app_eventsData.groupby(['app_id','event_id'])
print(len(app_eventsData),len(appDataGrpByObj.groups),len(eventsDataGrpByObj.groups),
      len(app_event_GrpByObj.groups),len(isActiveGrpByObj.groups),len(isInstalledGrpByObj.groups))
#No Single PK for the Table app_events: (app_id,event_id) together makes the PK
#2 Categorical Variables : is_active and is_installed
#2 Continuous but Identity Columns : app_id and event_id

99999 5504 3553 99999 2 1


In [15]:
app_labelsData=pd.read_csv("app_labels_subset.csv")
app_labelsData[0:1]

,app_id,label_id
0,7324884708820027918,251


In [16]:
app_idGrpByObj=app_labelsData.groupby('app_id')
label_idGrpByObj=app_labelsData.groupby('label_id')
print(len(app_idGrpByObj.groups),len(label_idGrpByObj.groups),len(app_labelsData))
#No PK for app_labels Table
app_labelsData.describe()
#label_id is between 251 to 691
#app_id is between -9.21e18 to 9.22e18

3502 31 4999


,app_id,label_id
count,4.999000e+03,4999.000000
mean,2.888705e+17,477.645929
std,5.277960e+18,127.849836
min,-9.216547e+18,251.000000
25%,-4.133423e+18,262.000000
50%,6.240908e+16,549.000000
75%,5.191946e+18,549.000000
max,9.220205e+18,691.000000


In [17]:
label_categoriesData=pd.read_csv("label_categories.csv")
label_categoriesData[0:1]

,label_id,category
0,1,NaN


In [18]:
label_categoriesData.describe()#label_id is from 1 to 1021
#Note I am working with Subset of Data: So there can be inconsistencies while trying to find exact matches
categoryGrpByObj=label_categoriesData.groupby('category')
labelidGrpByObj=label_categoriesData.groupby('label_id')
print(len(categoryGrpByObj.groups),len(labelidGrpByObj.groups),len(label_categoriesData))
#label_id is PK for label_categories table
print(categoryGrpByObj.groups)#Mostly there is On-to-One association between categort and label_id; However there
#are few excptions(1:N) like 'Financial Information': [143, 708, 921]
#Thought point: Not sure: If we will like to comeup with another category ourself with more generic categories
# In the subset of data number of categories are 835: Hopefullt their count is almost same for the entire dataset as well

835 930 930
{'game-Parkour Racing': [14], 'paid2': [750], 'game-stress reliever': [7], 'Cycling Sport': [429], 'Internet Securities': [927], 'Consumer orientation': [273], 'joke': [135], 'Xian Xia': [894], '2G': [571], 'Luxury brand': [328], 'Makeup': [333], 'Index Fund': [665], 'Corporate': [595], 'Commodity Futures': [689], 'flat': [536], 'Europe, the United States and Macao (Travel)': [858], 'Lenovo Cooperation': [820], 'phone tools': [220], 'show': [161, 257], 'Feature Pack': [331], 'Women': [288], 'navigation': [165, 896], 'Game willingness to pay': [466], 'Consumer preferences': [271], "Read the operator's": [736], 'Simulators': [884], 'Buy': [248], 'video': [160], 'BM': [874], 'paid4': [752], 'Chinese Classical Mythology': [81], 'the little sun': [767], 'operator 2': [565], 'Airline Alliances': [836], 'Beach landing game': [54], 'hospital': [374], 'Animation': [731, 797], '3 kindom game': [76], 'Collection process': [340], 'education outside class': [116], 'Hong Kong, Macao and 

In [19]:
phone_brand_device_modelData=pd.read_csv("phone_brand_device_model.csv")
phone_brand_device_modelData[0:5]

,device_id,phone_brand,device_model
0,-8890648629457979026,小米,红米
1,1277779817574759137,小米,MI 2
2,5137427614288105724,三星,Galaxy S4
3,3669464369358936369,SUGAR,时尚手机
4,-5019277647504317457,三星,Galaxy Note 2


In [21]:
chinestToEnglishDict={}
with open("ChineseToEnglishDict") as f:
    for line in f:
       (key, val) = line.split()
       chinestToEnglishDict[key] = val
print(chinestToEnglishDict)

{'富可视': 'fks', '朵唯': 'dowe', '海尔': 'haier', '黑米': 'heymi', '荣耀6': 'Glory6', '小杨树': 'smallt', '基伍': 'kiwu', '乐视': 'lshi', '语信': 'yuxin', '唯米': 'weimi', '中兴': 'ZTE', '米奇': 'micky', '聆韵': 'lingyun', '摩托罗拉': 'moto', '小米': 'millet', '夏新': 'panosonic', '诺基亚': 'nokia', '邦华': 'banghua', '布拉格': 'Prague', '海信': 'hisense', '米歌': 'mige', '酷比魔方': 'koobee', '欧比': 'obi', '三星': 'samsung', '糖葫芦': 'candy', '纽曼': 'newman', '梦米': 'mengmi', '神舟': 'hasse', '摩乐': 'mole', '普耐尔': 'pner', '西米': 'ximi', '酷派': 'cool', '谷歌': 'google', '乡米': 'xiangmi', '沃普丰': 'wpf', '德赛': 'desci', '联想': 'lenovo', '超级手机': 'SuperPhone', '尼比鲁': 'nibilu', '波导': 'bodao', '艾优尼': 'aiyouni', '华硕': 'asus', '天语': 'Ktouch', '爱派尔': 'ipair', '优米': 'youmi', '昂达': 'ada', '红米': 'Redrice', '魅族': 'meizu', '欧新': 'ouxin', '青橙': 'greeno', '飞秒': 'fs', '美图': 'meitu', '酷珀': 'kupo', '一加': 'oneplus', '糯米': 'nuomi', '维图': 'weitu', '中国移动': 'ccmc', '大可乐': 'bigcola', '金立': 'gionee', '时尚手机': 'FashionPhones', '易派': 'epai', '优购': 'yougo', '华为': 'huawei', 'E派': 'ep

In [26]:
def convertChineseToEnglish(x):
    convertedVal=chinestToEnglishDict.get(x)
    if convertedVal==None:
        return x
    return convertedVal
englishPhoneBrands=phone_brand_device_modelData.phone_brand.apply(convertChineseToEnglish)#Assign It back to DF
phone_brand_device_modelData.phone_brand=englishPhoneBrands
englishDeviceModels=phone_brand_device_modelData.device_model.apply(convertChineseToEnglish)#Assign It back to DF
phone_brand_device_modelData.device_model=englishDeviceModels

In [31]:
#Concatenating TrainFile with Device Information
train_device = pd.concat([trainFile, phone_brand_device_modelData], axis=1, join_axes=[trainFile.index])
train_device[0:4]

,device_id,gender,age,group,device_id,phone_brand,device_model
0,-8076087639492063270,M,35,M32-38,-8890648629457979026,millet,Redrice
1,-2897161552818060146,M,35,M32-38,1277779817574759137,millet,MI 2
2,-8260683887967679142,M,35,M32-38,5137427614288105724,samsung,Galaxy S4
3,-4938849341048082022,M,30,M29-31,3669464369358936369,SUGAR,FashionPhones


In [32]:
#Concatenating Train File With Events Information
train_device_events = pd.concat([train_device, eventsData], axis=1, join_axes=[train_device.index])
train_device_events[0:4]

,device_id,gender,age,group,device_id,phone_brand,device_model,event_id,device_id,timestamp,longitude,latitude
0,-8076087639492063270,M,35,M32-38,-8890648629457979026,millet,Redrice,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24
1,-2897161552818060146,M,35,M32-38,1277779817574759137,millet,MI 2,2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97
2,-8260683887967679142,M,35,M32-38,5137427614288105724,samsung,Galaxy S4,3,-4833982096941402721,2016-05-01 00:08:05,106.60,29.70
3,-4938849341048082022,M,30,M29-31,3669464369358936369,SUGAR,FashionPhones,4,-6815121365017318426,2016-05-01 00:06:40,104.27,23.28


In [35]:
#Collecting All APP Specific Information i.e. app info with label information
app_events_labels=pd.concat([app_eventsData,app_labelsData],axis=1,join_axes=[app_eventsData.index])
app_events_labels[0:3]

,event_id,app_id,is_installed,is_active,app_id,label_id
0,2,5927333115845830913,1,1,7.324885e+18,251
1,2,-5720078949152207372,1,0,-4.494217e+18,251
2,2,-1633887856876571208,1,0,6.058196e+18,406


In [36]:
app_events_labels_category=pd.concat([app_events_labels,label_categoriesData],axis=1,join_axes=[app_events_labels.index])
app_events_labels_category[0:3]

,event_id,app_id,is_installed,is_active,app_id,label_id,label_id,category
0,2,5927333115845830913,1,1,7.324885e+18,251,1,NaN
1,2,-5720078949152207372,1,0,-4.494217e+18,251,2,game-game type
2,2,-1633887856876571208,1,0,6.058196e+18,406,3,game-Game themes


In [37]:
#Concatenating Device Information with App Information
train_device_events_app=pd.concat([train_device_events,app_events_labels_category],
                                  axis=1,join_axes=[train_device_events.index])
train_device_events_app[0:4]

,device_id,gender,age,group,device_id,phone_brand,device_model,event_id,device_id,timestamp,longitude,latitude,event_id,app_id,is_installed,is_active,app_id,label_id,label_id,category
0,-8076087639492063270,M,35,M32-38,-8890648629457979026,millet,Redrice,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24,2,5927333115845830913,1,1,7.324885e+18,251,1,NaN
1,-2897161552818060146,M,35,M32-38,1277779817574759137,millet,MI 2,2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97,2,-5720078949152207372,1,0,-4.494217e+18,251,2,game-game type
2,-8260683887967679142,M,35,M32-38,5137427614288105724,samsung,Galaxy S4,3,-4833982096941402721,2016-05-01 00:08:05,106.60,29.70,2,-1633887856876571208,1,0,6.058196e+18,406,3,game-Game themes
3,-4938849341048082022,M,30,M29-31,3669464369358936369,SUGAR,FashionPhones,4,-6815121365017318426,2016-05-01 00:06:40,104.27,23.28,2,-653184325010919369,1,1,6.058196e+18,407,4,game-Art Style


In [38]:
train_device_events_app.to_csv("DeNormalizedTrainingData.csv")